# L3: Continuous Markov Process

In [ ]:

%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx

import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl

In [ ]:
from scipy.linalg import expm

In [ ]:
def get_filename(filename: str, lecture_id: int = 1, file_extension: str = '.png') -> str:
    return f"L{lecture_id}_{filename}{file_extension}"

outdir = '../figures/'
lecture_id = 3

In [ ]:
'''
------------------------------------------
            SETTINGS
------------------------------------------
'''
plt.style.use('fivethirtyeight')
plt.style.use('seaborn-v0_8-white')
plt.rcParams['font.family'] = 'PT Sans'
# plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 14
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['figure.titlesize'] = 12

dpi = 100

# 1. Two-state Markov chain example

In [ ]:
lmbda = 0.2
mu = 0.9

In [ ]:
states = ["Operational", "Failure"]
Q = np.array([
    [0,lmbda], 
    [mu, 0]
])
Q.shape

#### Check that it is a valid Q
Row-sum = 0 

In [ ]:
def check_valid_Q(Q: np.ndarray) -> bool:
    assert np.all(np.isclose(np.sum(Q,1),0)),np.sum(Q,1)
    return True

In [ ]:
check_valid_Q(Q)

Correct it by fixing diagonal entries

In [ ]:
for i in range(Q.shape[0]):
    rowSum = Q[i].sum() 
    Q[i,i] = - (rowSum - Q[i,i])
check_valid_Q(Q)
print("Check passed")

In [ ]:
filename = 'two_state_cmc'
filename = get_filename(filename,lecture_id=lecture_id)

outfile = filename
outfile

In [ ]:
def get_graph_from_Q(Q: np.ndarray,states: list, keep_selfloop: bool = False) -> nx.MultiDiGraph():
    G = nx.MultiDiGraph()
    assert Q.shape[0] == len(states)
    for start_idx, node_start in enumerate(states):
        for end_idx, node_end in enumerate(states):
            value = Q[start_idx][end_idx]
            if value != 0:
                G.add_edge(node_start,node_end, weight=value)

    if keep_selfloop == False:
        G.remove_edges_from(nx.selfloop_edges(G))
    return G

In [ ]:
fs = 20
color = 'salmon'

In [ ]:

plt.figure(figsize=(8,4))

# Using a matrix visualization

plt.subplot(1,2,1)
plt.imshow(Q, vmax=1,vmin=0, cmap='Blues')

for (j,i),label in np.ndenumerate(Q):
    plt.text(i,j,f"{label:.1f}",ha='center',va='center', c=color, fontsize = fs)
    plt.text(i,j,f"{label:.1f}",ha='center',va='center', c=color, fontsize = fs)
plt.axis('off')
plt.colorbar()

# Using a graph visualization
plt.subplot(1,2,2)
ax = plt.gca()

G = get_graph_from_Q(Q,states)

pos = nx.spring_layout(G, seed=10)

nx.draw_networkx_nodes(G, pos, node_size=1000, edgecolors='black', node_color='white',ax=ax)
nx.draw_networkx_labels(G, pos, font_size=12,ax=ax)

arc_rad = 0.2

edges = nx.draw_networkx_edges(G, pos, ax=ax, connectionstyle=f'arc3, rad = {arc_rad}', edge_cmap=cm.Blues, width=5,
    edge_color=[G[nodes[0]][nodes[1]][0]['weight'] for nodes in G.edges])

pc = mpl.collections.PatchCollection(edges, cmap=cm.Blues)

# ax = plt.gca()
ax.set_axis_off()
plt.colorbar(pc, ax=ax)
plt.tight_layout()
# plt.show()
if outfile is not None:
    
    plt.savefig(f"{outdir}{outfile}", dpi=dpi, format=None, metadata=None,
                bbox_inches='tight', pad_inches=0.1,
                facecolor='auto', edgecolor='auto',
                backend=None
                )
    print(f"Figure saved in {outdir}{outfile}")
        
plt.show()

# 2. Simulate a Markov Chain probability distribution

#### 2.1 Get $P$ from $Q$
Using:
$P(t) = e^{Qt}$

In [ ]:
def get_P(Q:np.ndarray,t: float) -> np.ndarray:
    '''
    Get P(t) = exp(Qt)
    '''
    return expm(Q * t)

#### Vary $t$ to get familiar with how $P(t)$ changes

In [ ]:
t = 1000
P = get_P(Q, t = t)

plt.figure(figsize=(4,4))
plt.imshow(P, vmax=1,vmin=0, cmap='Blues')

for (j,i),label in np.ndenumerate(P):
    plt.text(i,j,f"{label:.1f}",ha='center',va='center', c=color, fontsize = fs)
    plt.text(i,j,f"{label:.1f}",ha='center',va='center', c=color, fontsize = fs)
plt.axis('off')
plt.colorbar()

#### 2.2 Simulate using $s(t) = s(0)\, P(t)$

In [ ]:
def simulate(s0: np.ndarray, Q: np.ndarray,
             steps: list) -> np.ndarray:
    
    T = len(steps)
    out = np.zeros(shape=(T+1, len(s0)))
    out[0, :] = s0

    for i in range(1,T+1):
        P = get_P(Q, t = steps[i-1])
        out[i, :] = np.dot(out[i - 1, :], P)

    return out


#### Play with $s(0)=$`s0`

In [ ]:
time_steps = [0.01,0.05,0.1,0.5,1.0,2.0,5.0,10.,100.,1000]
time_steps.sort()
T = len(time_steps)+1 # number of timesteps to simulate

fig, ax = plt.subplots(1,T,figsize=(6 * T, 8))

# Set initial state
s0 = [0.01,0.99]  # Starting state
print(f"Starting state distribution = {s0}")

state_forecast = simulate(s0,Q,steps = time_steps)
for i in range(T):

    current_state = state_forecast[i]
    nx.draw_networkx_nodes(G, pos, node_size=3000, edgecolors='black', node_color=current_state,ax=ax[i],alpha=0.8, cmap=cm.Blues,vmax=1.,vmin=0)
    nx.draw_networkx_labels(G, pos, font_size=12, ax=ax[i])

    edges = nx.draw_networkx_edges(G, pos, ax=ax[i], connectionstyle=f'arc3, rad = {arc_rad}', edge_cmap=cm.Blues, width=5,
    edge_color=[G[nodes[0]][nodes[1]][0]['weight'] for nodes in G.edges])
    
    ax[i].set_axis_off()
    if i == 0:
        title = 't=0'
    else:
        title = f"t={time_steps[i-1]}"
    ax[i].set_title(title, fontsize=40)
    

Do you notice something odd or are you confused?

The use of the word `simulate` is probably not ideal and misleading...  
Here we were visualizing the behavior of $s(t)$.
But this is a probability distribution,  **not** and individual sample realization!   
Hence, this was not really a simulation of the process...  
It was useful to see how $s(t)$ changes depending on where you start, i.e. $s_0$, but it is probably more appropriate to visualize this with a heatmap instead.

**Q**: How can we instead simulate a sample realization?


### 2.3 Simulate a sample of the process
We can use what we learned about continuous-time Markov processes: 
1. Select a starting state $X(t=0)=i$
2. Stay there a time $\tau_0 \sim exp(q_i)$
3. Select the next state $j \sim \frac{q_{ij}}{q_i}$
4. Iterate


In [ ]:
def simulate_sample_cmc(x0: int, Q: np.ndarray,
             n_steps: list, prng: np.random.RandomState = None,
                       seed: int = 10) -> dict:

    if prng is None:
        prng = np.random.RandomState(seed=seed)

    N = Q.shape[0]
    assert x0 < N
    states = np.arange(N)
    
    time_spent_in_i = {i: 0 for i in states} # keep track of time spent in each state, for statistics
    out = np.zeros(shape=(n_steps+1)).astype(int)
    tau = np.zeros(shape=(n_steps+1)).astype(int)
    out[0] = x0
    tau[0] = prng.exponential(scale=1/(-Q[x0,x0]), size=1)[0] # time spent in x0
    time_spent_in_i[out[0]] += tau[0]
    for n in range(1,n_steps+1):

        p = Q[out[n-1]][states!=out[n-1]] # here we select the correct row of Q to get the transition probabilities, excluding the diagonal
        p /= p.sum()
        out[n] = prng.choice(states[states!=out[n-1]], p=p) # sample the next state j
    
        tau[n] = prng.exponential(scale=1/(-Q[out[n],out[n]]), size=1)[0] # time spent in x0
        time_spent_in_i[out[n]] += tau[n]

    return {'xs':out,'taus':tau, 'pis':time_spent_in_i}

Alternatively, you can sample $\tau_j$ from $\exp(q_{ij})$ for all $j\neq i$, and then select the one with the minimum value

In [ ]:
def simulate_sample_cmc_2(x0: int, Q: np.ndarray,
             n_steps: list, prng: np.random.RandomState = None,
                       seed: int = 10) -> dict:

    if prng is None:
        prng = np.random.RandomState(seed=seed)

    N = Q.shape[0]
    assert x0 < N
    states = np.arange(N)
    
    time_spent_in_i = {i: 0 for i in states} # keep track of time spent in each state, for statistics
    out = np.zeros(shape=(n_steps+1)).astype(int)
    tau = np.zeros(shape=(n_steps+1)).astype(int)
    out[0] = x0
    tau[0] = prng.exponential(scale=1/(-Q[x0,x0]), size=1)[0] # time spent in x0
    time_spent_in_i[out[0]] += tau[0]
    for n in range(1,n_steps+1):

        current_state = out[n-1]
        # Sample the taus in each state
        tau_j = [prng.exponential(scale=1/rate, size=1)[0] for rate in Q[current_state][:current_state]]
        tau_j += [1e10]  # An infinite sojourn to the same state, so that we do not select this
        tau_j += [prng.exponential(scale=1/rate, size=1)[0] for rate in Q[current_state][current_state + 1:]]
        tau_j = np.array(tau_j)
        
        # Identify the next state, selecting the min exp time
        out[n] = np.argmin(tau_j)
        tau[n] = np.min(tau_j)
        time_spent_in_i[out[n]] += tau[n]

    return {'xs':out,'taus':tau, 'pis':time_spent_in_i}

In [ ]:
seed = 10
prng = np.random.RandomState(seed=seed)

states = ["Operational", "Failure"]
assert len(states) == Q.shape[0]

In [ ]:
T = 100 # number of timesteps to simulate
N = Q.shape[0] 

# Set initial state
x0 = 0
print(f"Starting state x = {states[x0]}")

sample_realization = simulate_sample_cmc(x0,Q,n_steps = T,prng=prng)
ts = np.cumsum(sample_realization['taus'])  # compute successive sums to create a sequence of arrival times 
sample_realization['xs'],ts

In [ ]:
outfile = None

In [ ]:
plt.figure(figsize=(9,5))

plt.subplot(2,1,1)
plt.scatter(ts, sample_realization['xs'], s = 100, edgecolors='black')
plt.ylabel('Event happening')
plt.xlabel('t')
plt.yticks(np.arange(N),labels=states)

plt.subplot(2,1,2)
plt.step(ts,sample_realization['xs'])
plt.ylabel('X(t)')
plt.xlabel('t')
plt.yticks(np.arange(N),labels=states)

plt.tight_layout()
if outfile is not None:
    
    plt.savefig(f"{outdir}{outfile}", dpi=dpi, format=None, metadata=None,
                bbox_inches='tight', pad_inches=0.1,
                facecolor='auto', edgecolor='auto',
                backend=None
                )
    print(f"Figure saved in {outdir}{outfile}")


    

Let's check the stats about **how much time** was **spent** in each state.

Compare with $P(t)$ at large time:  
$\lim_{t \rightarrow \infty} P(t)$

In [ ]:

plt.figure(figsize=(12,4))

plt.subplot(1,3,1)

tot_time = sum(sample_realization['pis'].values())
y = np.array(list(sample_realization['pis'].values()))  / tot_time

plt.bar(states,y)
plt.xlabel('State',fontsize=fs)
plt.ylabel(f"% time spent in state",fontsize=fs)

pad = 0.1
for i,s in enumerate(states):
    plt.text(i - pad,y[i] - pad,f"{y[i]:.2f}",fontsize=fs,color='white')
plt.ylim([0,1])

plt.subplot(1,3,2)
bins, y = np.unique(sample_realization['xs'], return_counts=True)
y = y / y.sum()
plt.bar(bins,y)
plt.xlabel('State',fontsize=fs)
plt.ylabel(f"% jumps to state",fontsize=fs)
plt.ylim([0,1])

# pad = 0.1
# for i,s in enumerate(states):
#     plt.text(i - pad,y[i] - pad,f"{y[i]:.2f}",fontsize=fs,color='white')
# plt.ylim([0,1])

plt.subplot(1,3,3)

t = 1000
P_infty = get_P(Q, t = t)

plt.imshow(P_infty, vmax=1,vmin=0, cmap='Blues')

for (j,i),label in np.ndenumerate(P_infty):
    plt.text(i,j,f"{label:.2f}",ha='center',va='center', c=color, fontsize = fs)
    plt.text(i,j,f"{label:.2f}",ha='center',va='center', c=color, fontsize = fs)
plt.axis('off')
plt.colorbar()

plt.tight_layout()

But we can also compare with the average % of _sojurn_ times:

$\frac{\tau_j}{\sum_k \tau_k}$

In [ ]:
1 / (-np.diag(Q)) 

In [ ]:
1 / (-np.diag(Q)) / (1 / (-np.diag(Q)) ).sum()

In [ ]:
empirical_sojourn = np.zeros(len(states))
for i in range(len(states)):
    mask = sample_realization['xs'] == i
    empirical_sojourn[i] = sample_realization['taus'][mask].mean()
empirical_sojourn, empirical_sojourn/empirical_sojourn.sum() 

# 3. Check different ways of _computing_ $P$ from $Q$
Using:
$P(t) = e^{Qt}$

In [ ]:
Q

You can play with $t$, changing from small to large 

In [ ]:
t = 1.0

In [ ]:
print(f"P =\n")
get_P(Q, t = t)

### 3.1 Method 1: Using the Taylor expansion $e^{Qt} = \sum_{k=0}^{\infty}\frac{({Qt})^{k}}{k !}$

In [ ]:
from numpy.linalg import matrix_power
from math import factorial

In [ ]:
max_k = 2
P1 = sum([matrix_power(Q * t, k) / factorial(k) for k in range(max_k)])
P1

### 3.2 Method 2: Use the eigen-decomposition of $e^{Qt} = X \,\text{diag}(e^{\lambda_{k}t}) Y^{T}$

In [ ]:
eigenvalues_left, Y = np.linalg.eig(Q.T) #  left eigenvector (i.e. y.T @ Q = z * y.T)
eigenvalues_right, X = np.linalg.eig(Q) # right eigenvector
assert np.allclose(eigenvalues_left,eigenvalues_right)

D = np.diag(np.exp(eigenvalues_right * t)) # build diagonal matrix

P2 = np.matmul(np.matmul(X, D),Y.T) 
P2 / P2.sum(axis=1)


# 4. Find steady-state distribution

Solving an eigenvector equation.

#### 4.1 Method 1: Analytical solution for two-state example

TODO: homework

Result is:  
$\pi = \left(\frac{\mu}{\mu + \lambda},\frac{\lambda}{\mu + \lambda}\right)$

In [ ]:
den = lmbda + mu
pi = np.array([mu, lmbda]) / den
pi


#### 4.2 Method 2: Numerical solution

In [ ]:
# Calculate the steady-state distribution
eigenvalues, eigenvectors = np.linalg.eig(Q.T) 
steady_state = eigenvectors[:, np.isclose(eigenvalues, 0)]

# Normalize the steady-state distribution
steady_state = steady_state / steady_state.sum()

print(steady_state.real.flatten())

#### 4.3 Method 3: compare with $P$ at large time
$\lim_{t \rightarrow \infty} P(t)$

In [ ]:
t = 1000
get_P(Q, t = t)

# 5. How long do you expect on average to stay in each state?

This is the sojourn time $\tau$!  
$\{\tau_j\}_j$ are i.i.d. variables distributed with an exponential distribution with mean $1/ q_j$.

In [ ]:
1 / (-np.diag(Q))

# 6. Sampled Markov chain
Let's see how we can get discrete samples of a continuous chain.  
These are often more interpretable quantities, but we need to be careful in using them as "proxy" for a continuous process.  
Depending on the way we sample we can have (or not) properties that resemble the ones of the original continuous process.

#### 6.1 Let's use the Q as in exercise 10.8 (i)

In [ ]:
outfile = None

In [ ]:
lmbda = 1 / 1000 # 1/h
mu = 1 / 100 # 1/h
states = ['Both ok', 'One ok', 'None ok']

In [ ]:
Q = [
    [-2 * lmbda, 2 * lmbda, 0],
     [mu, -(lmbda+mu),lmbda],
     [0,2 * mu, -2* mu]
    ]
Q = np.array(Q)
Q.shape

#### First check that this is a valid Q

In [ ]:
check_valid_Q(Q)

####  Visualize the Q

In [ ]:

plt.figure(figsize=(10,4))

# Using a matrix visualization

plt.subplot(1,2,1)
plt.imshow(Q, vmax=Q.max(),vmin=0, cmap='Blues')

for (j,i),label in np.ndenumerate(Q):
    plt.text(i,j,f"{label:.3f}",ha='center',va='center', c=color, fontsize = fs)
    plt.text(i,j,f"{label:.3f}",ha='center',va='center', c=color, fontsize = fs)
plt.axis('off')
plt.colorbar()

# Using a graph visualization
plt.subplot(1,2,2)
ax = plt.gca()

G = get_graph_from_Q(Q,states)

pos = nx.spring_layout(G, seed=10)

nx.draw_networkx_nodes(G, pos, node_size=1000, edgecolors='black', node_color='white',ax=ax)
nx.draw_networkx_labels(G, pos, font_size=12,ax=ax)

arc_rad = 0.2

edges = nx.draw_networkx_edges(G, pos, ax=ax, connectionstyle=f'arc3, rad = {arc_rad}', edge_cmap=cm.Blues, width=5,
    edge_color=[G[nodes[0]][nodes[1]][0]['weight'] for nodes in G.edges])

pc = mpl.collections.PatchCollection(edges, cmap=cm.Blues)

# ax = plt.gca()
ax.set_axis_off()
plt.colorbar(pc, ax=ax)
plt.tight_layout()
# plt.show()
if outfile is not None:
    
    plt.savefig(f"{outdir}{outfile}", dpi=dpi, format=None, metadata=None,
                bbox_inches='tight', pad_inches=0.1,
                facecolor='auto', edgecolor='auto',
                backend=None
                )
    print(f"Figure saved in {outdir}{outfile}")
        
plt.show()

## 6.2 Method 1: use the sampled-time routine of 10.4.2

#### 6.2.1 Check average sojourn times
We need to guarantee that transition probabilities are valid probabilities first:  
$\Delta t < 1/ \max_i q_i$

In [ ]:
t_soj = 1 / (-np.diag(Q)) # in hours

t_soj

In [ ]:
np.min(1 / (-np.diag(Q)))

Let's use this as an upper bound for our choice of $\Delta t$

In [ ]:
delta_t = 50 # h

Let's now build $P$ using:

- $P_{ij} = q_{ij} \Delta t \quad, \quad$ for $j \neq i$
- $P_{ii} = 1 - q_{i} \Delta t$  


In [ ]:
P = Q * delta_t

np.fill_diagonal(P,1-(-np.diag(Q)) * delta_t)

In [ ]:

plt.figure(figsize=(10,4))

# Using a matrix visualization

plt.subplot(1,2,1)
plt.imshow(P, vmax=P.max(),vmin=0, cmap='Blues')

for (j,i),label in np.ndenumerate(P):
    plt.text(i,j,f"{label:.3f}",ha='center',va='center', c=color, fontsize = fs)
    plt.text(i,j,f"{label:.3f}",ha='center',va='center', c=color, fontsize = fs)
plt.axis('off')
plt.colorbar()

# Using a graph visualization
plt.subplot(1,2,2)
ax = plt.gca()

G = get_graph_from_Q(P,states, keep_selfloop=True)

pos = nx.spring_layout(G, seed=10)

nx.draw_networkx_nodes(G, pos, node_size=1000, edgecolors='black', node_color='white',ax=ax)
nx.draw_networkx_labels(G, pos, font_size=12,ax=ax)

arc_rad = 0.2

edges = nx.draw_networkx_edges(G, pos, ax=ax, connectionstyle=f'arc3, rad = {arc_rad}', edge_cmap=cm.Blues, width=5,
    edge_color=[G[nodes[0]][nodes[1]][0]['weight'] for nodes in G.edges])

pc = mpl.collections.PatchCollection(edges, cmap=cm.Blues)

# ax = plt.gca()
ax.set_axis_off()
plt.colorbar(pc, ax=ax)
plt.tight_layout()
# plt.show()
if outfile is not None:
    
    plt.savefig(f"{outdir}{outfile}", dpi=dpi, format=None, metadata=None,
                bbox_inches='tight', pad_inches=0.1,
                facecolor='auto', edgecolor='auto',
                backend=None
                )
    print(f"Figure saved in {outdir}{outfile}")
        
plt.show()

#### Check it is a valid stochastic matrix

In [ ]:
assert np.all(np.isclose(np.sum(P,1),1)),np.sum(P,1)

In [ ]:
# Simulate transitions
def next_state(current_state, transition_matrix):
    np.all(np.isclose(np.sum(transition_matrix,1),1))
    N = transition_matrix.shape[0]
    assert 0 <= current_state < N
    return np.random.choice(np.arange(N), p=transition_matrix[current_state])


current_state = 0  # Starting state
days = 20 # number of timesteps to simulate
state_forecast = [states[current_state]]

fig, ax = plt.subplots(1,days,figsize=(6 * days, 5))

print(f"Starting state = {states[current_state]}")
for i in range(days):

    old_state = current_state
    node_color = ['white' for i in range(len(states))]
    node_color[old_state] = 'r'
    current_state = next_state(current_state, P)
    state_forecast.append(states[current_state])

    nx.draw_networkx_nodes(G, pos, node_size=3000, edgecolors='black', node_color=node_color,ax=ax[i],alpha=0.8)
    nx.draw_networkx_labels(G, pos, font_size=12, ax=ax[i])

    edge_color = ['black' for e in G.edges()]
    width = [1 for e in G.edges]
    for idx, (u,v) in enumerate(G.edges()):
        if (u == states[old_state]) & (v == states[current_state]):
            edge_color[idx] = 'r'
            width[idx] = 10
    edges = nx.draw_networkx_edges(G, pos, ax=ax[i], connectionstyle=f'arc3, rad = {arc_rad}', edge_cmap=cm.Blues, width=width,
                edge_color=edge_color, arrows=True, arrowsize=10)
    
    ax[i].set_axis_off()
    

print(state_forecast[1:])

## 6.3 Method 2: using embedded Markov chain

#### Let's create the transition matrix using:

$V_{ij} = q_{ij}/q_i$

In [ ]:
V = np.einsum('ij,i->ij',Q, -1/np.diag(Q))
np.fill_diagonal(V,0)
V

Let's check that it is a valid transition probability

In [ ]:
assert np.all(np.isclose(np.sum(V,1),1)),np.sum(V,1)

In [ ]:

plt.figure(figsize=(10,4))

# Using a matrix visualization

plt.subplot(1,2,1)
plt.imshow(V, vmax=V.max(),vmin=0, cmap='Blues')

for (j,i),label in np.ndenumerate(V):
    plt.text(i,j,f"{label:.3f}",ha='center',va='center', c=color, fontsize = fs)
    plt.text(i,j,f"{label:.3f}",ha='center',va='center', c=color, fontsize = fs)
plt.axis('off')
plt.colorbar()

# Using a graph visualization
plt.subplot(1,2,2)
ax = plt.gca()

G = get_graph_from_Q(V,states, keep_selfloop=True)

pos = nx.spring_layout(G, seed=10)

nx.draw_networkx_nodes(G, pos, node_size=1000, edgecolors='black', node_color='white',ax=ax)
nx.draw_networkx_labels(G, pos, font_size=12,ax=ax)

arc_rad = 0.2

edges = nx.draw_networkx_edges(G, pos, ax=ax, connectionstyle=f'arc3, rad = {arc_rad}', edge_cmap=cm.Blues, width=5,
    edge_color=[G[nodes[0]][nodes[1]][0]['weight'] for nodes in G.edges])

pc = mpl.collections.PatchCollection(edges, cmap=cm.Blues)

# ax = plt.gca()
ax.set_axis_off()
plt.colorbar(pc, ax=ax)
plt.tight_layout()
# plt.show()
if outfile is not None:
    
    plt.savefig(f"{outdir}{outfile}", dpi=dpi, format=None, metadata=None,
                bbox_inches='tight', pad_inches=0.1,
                facecolor='auto', edgecolor='auto',
                backend=None
                )
    print(f"Figure saved in {outdir}{outfile}")
        
plt.show()

We can now use these transitions to simulate a discrete-time sample of the process  
(using simulation similar to the one contained in notebook L2)

In [ ]:
# Simulate transitions
def next_state(current_state, transition_matrix):
    np.all(np.isclose(np.sum(transition_matrix,1),1))
    N = transition_matrix.shape[0]
    assert 0 <= current_state < N
    return np.random.choice(np.arange(N), p=transition_matrix[current_state])


current_state = 0  # Starting state
days = 20 # number of timesteps to simulate
state_forecast = [states[current_state]]

fig, ax = plt.subplots(1,days,figsize=(6 * days, 5))

print(f"Starting state = {states[current_state]}")
for i in range(days):

    old_state = current_state
    node_color = ['white' for i in range(len(states))]
    node_color[old_state] = 'r'
    current_state = next_state(current_state, V)
    state_forecast.append(states[current_state])

    nx.draw_networkx_nodes(G, pos, node_size=3000, edgecolors='black', node_color=node_color,ax=ax[i],alpha=0.8)
    nx.draw_networkx_labels(G, pos, font_size=12, ax=ax[i])

    edge_color = ['black' for e in G.edges()]
    width = [1 for e in G.edges]
    for idx, (u,v) in enumerate(G.edges()):
        if (u == states[old_state]) & (v == states[current_state]):
            edge_color[idx] = 'r'
            width[idx] = 10
    edges = nx.draw_networkx_edges(G, pos, ax=ax[i], connectionstyle=f'arc3, rad = {arc_rad}', edge_cmap=cm.Blues, width=width,
                edge_color=edge_color, arrows=True, arrowsize=10)
    
    ax[i].set_axis_off()
    

print(state_forecast[1:])

## 6.4 Find steady state

#### 6.4.1 Method 1: from the original continuous process, i.e. using Q

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(Q.T) 
steady_state = eigenvectors[:, np.isclose(eigenvalues, 0)]

# Normalize the steady-state distribution
steady_state = steady_state / steady_state.sum()

print(steady_state.real.flatten())

#### 6.4.2 Method 2: using the sampled-time discrete process

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(P.T)
steady_state = eigenvectors[:, np.isclose(eigenvalues, 1)]

# Normalize the steady-state distribution
steady_state = steady_state / steady_state.sum()

print(steady_state.real.flatten())

#### 6.4.3 Method 3: using the embedded discrete process

$\pi_i = (v_i/q_i)/\sum_j (v_j/q_j)$

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(V.T)
steady_state = eigenvectors[:, np.isclose(eigenvalues, 1)]

# Normalize the steady-state distribution
steady_state = steady_state / steady_state.sum()
v = steady_state.real.flatten()
print(v)

In [ ]:
pi_v = v / (-np.diag(Q))
pi_v /= pi_v.sum()
pi_v